In [1]:
pip install psycopg2-binary

Note: you may need to restart the kernel to use updated packages.


In [2]:
#for databricks
import pandas as pd
import xml.etree.ElementTree as ET
from sqlalchemy import create_engine
import os
from os import listdir
from os.path import isfile, join
import re
import zipfile
import pyspark
from pyspark.sql import SparkSession

In [3]:
#Create SparkSession
spark = SparkSession.builder.master("local[1]").appName("data-gurus").getOrCreate()
print(spark)

In [4]:
# for processing all the xml files in the dataset folder
def names_of_xml_files(dataset_path):
    '''

    :param dataset_path: of zip file
    :return: final list of files
    '''
    try:

        zf = zipfile.ZipFile(dataset_path, "r")

        xml_files = []
        for file in zf.namelist():
            if file.endswith(".xml"):
                xml_files.append(os.path.join(file))

        # print(xml_files)

        new_list = []
        for file in xml_files:
            file = file[:-4]
            new_list.append(file)

        list_changed = [f.replace("_", "") for f in new_list]
        for i in range(0, 10):
            list_changed = [f.replace(str(i), "") for f in list_changed]

        final_list_of_files = list(set(list_changed))
        return final_list_of_files
    except Exception as e:
        print("Error while capturing file names :", e)
        pass

In [5]:
# give path of dataset and file name without the number
def files_mask(dataset_path, filename):
    '''
    
    :param dataset_path: provide dataset path of zip folder
    :param filename: file name without extension type
    :return: matching list with the same kind of files
    '''
    # get all files names in the datasetfolder

    zf = zipfile.ZipFile(dataset_path, "r")
    all_files = []
    for file in zf.namelist():
        if file.endswith(".xml"):
            #[5:] removes DATA/ string from the beginning
            all_files.append(file)

    #all_files = [f for f in listdir(dataset_path) if isfile(join(dataset_path, f))]

    regex_pattern = ".*" + filename + ".*"

    match_list = []
    for i in range(0, len(all_files)):
        match = re.findall(regex_pattern, all_files[i])
        if not match:
            # if list is empty
            pass
        else:
            match_list.append(match[0])

    # print("Required Files in the Dataset: ", match_list)
    return match_list

In [6]:
def xml_to_dataframe_converter(dataset_path, xml_file):
    '''
    
    :param dataset_path:zip folder path 
    :param xml_file: full file name with the extension
    :return: dataframe
    '''

    zf = zipfile.ZipFile(dataset_path, "r")

    #if xml_file in zf.namelist():
    xml_file_open = zf.open(xml_file)
    tree = ET.parse(xml_file_open)
    root = tree.getroot()
    
    my_dict = {}
    for elem in root:
        for sub_elem in elem:
            # check if key already exists in the dictionary
            if my_dict.__contains__(sub_elem.tag):
                my_dict[sub_elem.tag].append(sub_elem.text)
            # else create the new key and append
            else:
                my_dict[sub_elem.tag] = []
                my_dict[sub_elem.tag].append(sub_elem.text)
             

    df = pd.DataFrame.from_dict(my_dict, orient='index').transpose()

    #print(df.dtypes)
    return df

In [7]:
# Spark data dictionary for each file type 
# TODO: please fill this schema dictionary for all files types, and check download
from pyspark.sql.types import StructType, StructField, StringType, TimestampType, LongType, DoubleType
schemaDict = {'anlageneegsolar': StructType([StructField('EegMaStRNummer', StringType(), True), 
                                             StructField('Registrierungsdatum', TimestampType(), True), 
                                             StructField('Zuschlagsnummer', StringType(), True), 
                                             StructField('MieterstromErsteZuordnungZuschlag', TimestampType(), True), 
                                             StructField('InstallierteLeistung', DoubleType(), True), 
                                             StructField('AusschreibungZuschlag', DoubleType(), True), 
                                             StructField('AnlagenkennzifferAnlagenregister_nv', LongType(), True), 
                                             StructField('ZugeordneteGebotsmenge', DoubleType(), True), 
                                             StructField('AnlageBetriebsstatus', LongType(), True), 
                                             StructField('DatumLetzteAktualisierung', StringType(), True), 
                                             StructField('RegistrierungsnummerPvMeldeportal_nv', LongType(), True), 
                                             StructField('VerknuepfteEinheitenMaStRNummern', StringType(), True), 
                                             StructField('MieterstromMeldedatum', TimestampType(), True), 
                                             StructField('EegInbetriebnahmedatum', TimestampType(), True), 
                                             StructField('AnlagenschluesselEeg', StringType(), True), 
                                             StructField('InanspruchnahmeZahlungNachEeg', DoubleType(), True), 
                                             StructField('AnlagenkennzifferAnlagenregister', StringType(), True), 
                                             StructField('MieterstromZugeordnet', DoubleType(), True), 
                                             StructField('RegistrierungsnummerPvMeldeportal', StringType(), True)]),
              'anlageneegspeicher': StructType([StructField('EegMaStRNummer', StringType(), True), 
                                                StructField('Registrierungsdatum', TimestampType(), True), 
                                                StructField('EegInbetriebnahmedatum', TimestampType(), True), 
                                                StructField('VerknuepfteEinheitenMaStRNummern', StringType(), True), 
                                                StructField('DatumLetzteAktualisierung', TimestampType(), True)]), 
              
              'anlagenstromspeicher': StructType([StructField('MaStRNummer', StringType(), True), 
                                                  StructField('DatumLetzteAktualisierung', TimestampType(), True), 
                                                  StructField('NutzbareSpeicherkapazitaet', DoubleType(), True), 
                                                  StructField('AnlageBetriebsstatus', LongType(), True), 
                                                  StructField('VerknuepfteEinheitenMaStRNummern', StringType(), True), 
                                                  StructField('Registrierungsdatum', TimestampType(), True)]),
              
              'einheitenstromspeicher': StructType([StructField('DatumLetzteAktualisierung', TimestampType(), True), 
                                                    StructField('GeplantesInbetriebnahmedatum', TimestampType(), True), 
                                                    StructField('Energietraeger', LongType(), True), 
                                                    StructField('NameStromerzeugungseinheit', StringType(), True), 
                                                    StructField('Ort', StringType(), True), 
                                                    StructField('Notstromaggregat', DoubleType(), True), 
                                                    StructField('Breitengrad', DoubleType(), True), 
                                                    StructField('Kraftwerksnummer_nv', LongType(), True), 
                                                    StructField('Registrierungsdatum', TimestampType(), True), 
                                                    StructField('Gemeindeschluessel', DoubleType(), True), 
                                                    StructField('BestandteilGrenzkraftwerk', DoubleType(), True), 
                                                    StructField('AnlagenbetreiberMastrNummer', StringType(), True), 
                                                    StructField('ZugeordnenteWirkleistungWechselrichter', DoubleType(), True), 
                                                    StructField('Land', LongType(), True), 
                                                    StructField('WeicDisplayName', StringType(), True), 
                                                    StructField('AcDcKoppelung', DoubleType(), True), 
                                                    StructField('Laengengrad', DoubleType(), True), 
                                                    StructField('PumpbetriebKontinuierlichRegelbar', DoubleType(), True), 
                                                    StructField('GenMastrNummer', StringType(), True), 
                                                    StructField('Gemarkung', StringType(), True), 
                                                    StructField('EinheitSystemstatus', LongType(), True), 
                                                    StructField('EinheitBetriebsstatus', LongType(), True), 
                                                    StructField('Einspeisungsart', DoubleType(), True), 
                                                    StructField('Hausnummer_nv', DoubleType(), True), 
                                                    StructField('Postleitzahl', StringType(), True), 
                                                    StructField('NichtVorhandenInMigriertenEinheiten', LongType(), True), 
                                                    StructField('Technologie', DoubleType(), True), 
                                                    StructField('Kraftwerksnummer', StringType(), True), 
                                                    StructField('Pumpspeichertechnologie', DoubleType(), True), 
                                                    StructField('Batterietechnologie', DoubleType(), True), 
                                                    StructField('FernsteuerbarkeitDv', DoubleType(), True), 
                                                    StructField('NetzbetreiberpruefungStatus', LongType(), True), 
                                                    StructField('DatumEndgueltigeStilllegung', TimestampType(), True), 
                                                    StructField('DatumDesBetreiberwechsels', TimestampType(), True), 
                                                    StructField('FernsteuerbarkeitDr', DoubleType(), True), 
                                                    StructField('DatumBeginnVoruebergehendeStilllegung', TimestampType(), True), 
                                                    StructField('NetzbetreiberpruefungDatum', TimestampType(), True), 
                                                    StructField('Weic', StringType(), True), 
                                                    StructField('EegMaStRNummer', StringType(), True), 
                                                    StructField('SpeMastrNummer', StringType(), True), 
                                                    StructField('DatumRegistrierungDesBetreiberwechsels', TimestampType(), True), 
                                                    StructField('Bundesland', DoubleType(), True), 
                                                    StructField('AnschlussAnHoechstOderHochSpannung', DoubleType(), True), 
                                                    StructField('Einsatzort', DoubleType(), True), 
                                                    StructField('Einsatzverantwortlicher', DoubleType(), True), 
                                                    StructField('PumpbetriebLeistungsaufnahme', DoubleType(), True), 
                                                    StructField('EegAnlagentyp', LongType(), True), 
                                                    StructField('LokationMaStRNummer', StringType(), True), 
                                                    StructField('FlurFlurstuecknummern', StringType(), True), 
                                                    StructField('Inbetriebnahmedatum', TimestampType(), True), 
                                                    StructField('StrasseNichtGefunden', DoubleType(), True), 
                                                    StructField('Bruttoleistung', DoubleType(), True), 
                                                    StructField('Adresszusatz', StringType(), True), 
                                                    StructField('Weic_nv', LongType(), True), 
                                                    StructField('Gemeinde', StringType(), True), 
                                                    StructField('Landkreis', StringType(), True), 
                                                    StructField('HausnummerNichtGefunden', StringType(), True), 
                                                    StructField('EinheitMastrNummer', StringType(), True), 
                                                    StructField('Nettonennleistung', DoubleType(), True), 
                                                    StructField('Hausnummer', StringType(), True), 
                                                    StructField('FernsteuerbarkeitNb', DoubleType(), True), 
                                                    StructField('DatumWiederaufnahmeBetrieb', TimestampType(), True), 
                                                    StructField('Strasse', StringType(), True), 
                                                    StructField('NettonennleistungDeutschland', DoubleType(), True)]),

              'einheitensolar': StructType([StructField('EinheitMastrNummer', StringType(), True),
                                            StructField('DatumLetzteAktualisierung', StringType(), True),
                                            StructField('LokationMaStRNummer', StringType(), True),
                                            StructField('NetzbetreiberpruefungStatus', StringType(), True), 
                                            StructField('NetzbetreiberpruefungDatum', StringType(), True), 
                                            StructField('AnlagenbetreiberMastrNummer', StringType(), True), 
                                            StructField('Land', StringType(), True), 
                                            StructField('Bundesland', LongType(), True), 
                                            StructField('Landkreis', StringType(), True), 
                                            StructField('Gemeinde', LongType(), True), 
                                            StructField('Gemeindeschluessel', LongType(), True), 
                                            StructField('Postleitzahl', LongType(), True), 
                                            StructField('Ort', StringType(), True), 
                                            StructField('Registrierungsdatum', TimestampType(), True), 
                                            StructField('Inbetriebnahmedatum', StringType(), True), 
                                            StructField('EinheitSystemstatus', LongType(), True), 
                                            StructField('EinheitBetriebsstatus', LongType(), True), 
                                            StructField('NichtVorhandenInMigriertenEinheiten', LongType(), True), 
                                            StructField('NameStromerzeugungseinheit', StringType(), True), 
                                            StructField('Weic_nv', LongType(), True), 
                                            StructField('Kraftwerksnummer_nv', LongType(), True), 
                                            StructField('Energietraeger', LongType(), True), 
                                            StructField('Bruttoleistung', LongType(), True), 
                                            StructField('Nettonennleistung', LongType(), True), 
                                            StructField('FernsteuerbarkeitNb', LongType(), True), 
                                            StructField('Einspeisungsart', LongType(), True), 
                                            StructField('ZugeordneteWirkleistungWechselrichter', LongType(), True), 
                                            StructField('GemeinsamerWechselrichterMitSpeicher', LongType(), True), 
                                            StructField('AnzahlModule', LongType(), True), 
                                            StructField('Lage', LongType(), True), 
                                            StructField('Leistungsbegrenzung', LongType(), True), 
                                            StructField('EinheitlicheAusrichtungUndNeigungswinkel', LongType(), True), 
                                            StructField('Hauptausrichtung', StringType(), True), 
                                            StructField('HauptausrichtungNeigungswinkel', LongType(), True), 
                                            StructField('EegMaStRNummer', StringType(), True), 
                                            StructField('Nutzungsbereich', LongType(), True), 
                                            StructField('Nebenausrichtung', LongType(), True), 
                                            StructField('NebenausrichtungNeigungswinkel', LongType(), True), 
                                            StructField('Gemarkung', StringType(), True), 
                                            StructField('FlurFlurstuecknummern', StringType(), True), 
                                            StructField('GeplantesInbetriebnahmedatum', TimestampType(), True), 
                                            StructField('Strasse', StringType(), True), 
                                            StructField('StrasseNichtGefunden', LongType(), True), 
                                            StructField('Hausnummer', StringType(), True), 
                                            StructField('Hausnummer_nv', LongType(), True), 
                                            StructField('HausnummerNichtGefunden', LongType(), True), 
                                            StructField('Laengengrad', LongType(), True), 
                                            StructField('Breitengrad', LongType(), True), 
                                            StructField('InAnspruchGenommeneFlaeche', LongType(), True), 
                                            StructField('Adresszusatz', StringType(), True), 
                                            StructField('FernsteuerbarkeitDv', LongType(), True), 
                                            StructField('FernsteuerbarkeitDr', LongType(), True), 
                                            StructField('ArtDerFlaecheIds', StringType(), True), 
                                            StructField('DatumDesBetreiberwechsels', TimestampType(), True), 
                                            StructField('DatumRegistrierungDesBetreiberwechsels', TimestampType(), True), 
                                            StructField('DatumBeginnVoruebergehendeStilllegung', TimestampType(), True), 
                                            StructField('AnschlussAnHoechstOderHochSpannung', LongType(), True),
                                            StructField('DatumEndgueltigeStilllegung', TimestampType(), True),
                                            StructField('InAnspruchGenommeneAckerflaeche', LongType(), True),
                                            StructField('GenMastrNummer', LongType(), True),
                                            StructField('Einsatzverantwortlicher', LongType(), True),
                                            StructField('DatumWiederaufnahmeBetrieb', TimestampType(), True)])}
#print(schemaDict)

In [8]:
import warnings
warnings.filterwarnings('ignore') 

#new main
def main():
    # change dataset path
    #dataset_path = "G:\DATA GURUS WORK\AAAA\"
    dataset_path = "/home/jovyan/work/AAAA.zip"

    #file_names = ['AnlagenEegSolar', 'AnlagenEegSpeicher', 'AnlagenStromSpeicher', 'EinheitenStromSpeicher', 'EinheitenSolar']
    file_names = ['AnlagenEegSpeicher']

    for i in range(0, len(file_names)):
    
        file_name = file_names[i]
        
        spark.sql(f"drop table if exists marktstammdaten_{file_name.lower()}")

        print("Current File Type : ", file_name)

        xml_files_list = files_mask(dataset_path, file_name)
        print("XML FILES LIST: ", xml_files_list)

        #print(schemaDict[file_name.lower()])      #TODO: delete it
        
        for xml_file in xml_files_list:

            print(f"Processing : {xml_file}")
            data = xml_to_dataframe_converter(dataset_path,xml_file)
            # Create DataFrame
            #df = pd.DataFrame(data)
            #creating spark data frame            
            sdf = spark.createDataFrame(data, schema=schemaDict[file_name.lower()] if file_name.lower() in schemaDict.keys() else None)
            
            # change database table name
            table_name = file_name.lower()
            table_name = "marktstammdaten_" + table_name

            try:
                sdf.write.mode("append").saveAsTable(table_name)
            except Exception as e:
                print("Exception occurred")
                print(e)
                
    print("Finished processing!!!!")
                
    

In [9]:

if __name__ == "__main__":
    #main()
    main()


Current File Type :  AnlagenEegSpeicher
XML FILES LIST:  ['AAAA/AnlagenEegSpeicher_2.xml', 'AAAA/AnlagenEegSpeicher_3.xml', 'AAAA/AnlagenEegSpeicher_4.xml', 'AAAA/AnlagenEegSpeicher_5.xml']
Processing : AAAA/AnlagenEegSpeicher_2.xml


TypeError: field Registrierungsdatum: TimestampType() can not accept object '2021-07-15T04:49:24.4362156' in type <class 'str'>

In [ ]:
aes = spark.sql("select * from marktstammdaten_einheitensolar")
print(aes.count())
print(aes.schema)
aes.show()